# Fine-tuning Bloomz Seq2Seq Model on Botpress Dataset

## Development Environment and Permissions

### Installation

In [ ]:
!pip install accelerate nvidia-ml-py3 datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

### View GPU utilization

In [ ]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [ ]:
print_gpu_utilization()

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
import pandas as pd
import os
os.environ["WANDB_DISABLED"]="true"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
model_checkpoint = "bigscience/mt0-large"

# Load the fine-tuning dataset

In [ ]:
import json

with open('/content/ob-loose-jun28-sm.jsonl', 'r') as json_file:
    jsonl = json_file.readlines()

In [ ]:
from datasets import load_dataset, load_metric
raw_datasets = load_dataset("wmt16", "ro-en")
metric = load_metric("sacrebleu")

In [ ]:
# Format the data in the required format
data = [{'sequences':json.loads(t)} for t in jsonl]

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.1)

validation, test = train_test_split(test, test_size=0.5)


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.1)

validation, test = train_test_split(test, test_size=0.5)

import pandas as pd

train_df = pd.DataFrame(train)
validation_df = pd.DataFrame(validation)
test_df = pd.DataFrame(test)

In [ ]:
train_df.to_json(path_or_buf='/content/ob-loose-jun28-sm_train.jsonl', orient='records', lines=True)
validation_df.to_json(path_or_buf='/content/ob-loose-jun28-sm_validation.jsonl', orient='records', lines=True)
test_df.to_json(path_or_buf='/content/ob-loose-jun28-sm_test.jsonl', orient='records', lines=True)

In [ ]:
from datasets import load_dataset
base_path = '/content/ob-loose-jun28-sm_'
raw_datasets = load_dataset("json", data_files={"train": base_path + "train.jsonl", "validation": base_path + "validation.jsonl", "test": base_path + "test.jsonl"})

In [ ]:
raw_datasets

In [ ]:
raw_datasets["train"][0]

## Preprocessing

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
prefix = ""
max_input_length = 128
max_target_length = 128
source = "prompt"
target = "completion"

def preprocess_function(examples):
    inputs = [prefix + ex[source] for ex in examples["sequences"]]
    targets = [ex[target] for ex in examples["sequences"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

# Fine-tuning the Bloomz model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 1
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source}-to-{target}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    gradient_checkpointing=True,
    fp16=True
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!export PYTORCH_CUDA_ALLOC_CONF=garbage_collection_threshold:0.6,max_split_size_mb:128

In [ ]:
trainer.train()

# Save the fine-tuned model

In [ ]:
import os
for dirname, _, filenames in os.walk('/content/mt0-small-finetuned-prompt-to-completion'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
finetuned_model_name = model_checkpoint + "_finetuned"
trainer.save_model(finetuned_model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(finetuned_model_name)

In [ ]:
src_text = raw_datasets['test'][2]['sequences']['prompt']
tgt_text = raw_datasets['test'][2]['sequences']['completion']
print(src_text)
print(tgt_text)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(finetuned_model_name)
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]